# Build baseline tfrs model 

Look inside of `./two_tower_src/` for the source code and model code

In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

In [2]:
# !pip install tensorflow-recommenders==0.6.0 --user

In [3]:
TF_GPU_THREAD_MODE='gpu_private'
TF_GPU_ALLOCATOR='cuda_malloc_async'

In [4]:
?tf.data.experimental.OptimizationOptions

Object `tf.data.experimental.OptimizationOptions` not found.


In [5]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

from two_tower_src import two_tower as tt

2022-10-05 16:52:20.217049: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 16:52:21.039165: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 733 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


## Create Dataset for local training and testing

### Playlist dataset

In [6]:
batch_size = 2048*12

train_dir = 'spotify-beam-v3'
train_dir_prefix = 'v6/valid_last_5/'

valid_dir = 'spotify-beam-v3'
valid_dir_prefix = 'v6/valid_last_5/'

client = storage.Client()

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
options.experimental_optimization.map_and_batch_fusion = False


train_files = []
for blob in client.list_blobs(f'{train_dir}', prefix=f'{train_dir_prefix}', delimiter="/"):
    train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

train_dataset = tf.data.Dataset.from_tensor_slices(train_files).prefetch(
    tf.data.AUTOTUNE,
).with_options(options)
train_dataset = train_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False,
).map(
    tt.parse_tfrecord,
    num_parallel_calls=tf.data.AUTOTUNE,
).batch(
    batch_size 
).prefetch(
    tf.data.AUTOTUNE,
)


valid_files = []
for blob in client.list_blobs(f'{valid_dir}', prefix=f'{valid_dir_prefix}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(
    tf.data.AUTOTUNE,
).with_options(options)
valid_dataset = valid_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False,
).map(
    tt.parse_tfrecord,
    num_parallel_calls=tf.data.AUTOTUNE,
      ).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
)

# Local Training

In [7]:
layer_sizes=[256,128]
model = tt.TheTwoTowers(layer_sizes)

model.compile(optimizer=tf.keras.optimizers.Adagrad(0.01))

2022-10-05 16:52:38.143829: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.08GiB (rounded to 1158294016)requested by op RandomUniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-10-05 16:52:38.143873: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-10-05 16:52:38.143884: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 8, Chunks in use: 8. 2.0KiB allocated for chunks. 2.0KiB in use in bin. 49B client-requested in use in bin.
2022-10-05 16:52:38.143891: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-10-05 16:52:38.143897: I tensorflow/core/co

ResourceExhaustedError: Exception encountered when calling layer "candidate__track__model" (type Candidate_Track_Model).

in user code:

    File "/home/jupyter/spotify_mpd_two_tower/two_tower_src/two_tower.py", line 647, in call  *
        all_embs = tf.concat(
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 1922, in random_uniform
        seed=self.make_legacy_seed())

    ResourceExhaustedError: Exception encountered when calling layer "track_uri_can_emb_model" (type Sequential).
    
    OOM when allocating tensor with shape[2262293,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(None,), dtype=string)
      • training=None
      • mask=None


Call arguments received:
 

In [ ]:
## Quick look at the layers
print("Playlist (query) Tower:")

for i, l in enumerate(model.query_tower.layers):
    print(i, l.name)

In [ ]:
print("Track (candidate) Tower:")
for i, l in enumerate(model.candidate_tower.layers):
    print(i, l.name)

### Local training for one Epoch

In [ ]:
import time

In [ ]:
%%timeit -n 1 -r 1


NUM_EPOCHS = 1
start_time = time.time()
layer_history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    validation_freq=1,
    epochs=NUM_EPOCHS,
    # steps_per_epoch=10,
    # callbacks=tensorboard_cb,
    # verbose=0
)

print(f"Training for {NUM_EPOCHS} epoch")
accuracy = layer_history.history["factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top 100 categorical accuracy: {accuracy}")